# SAS 프로그램 결과

In [83]:
import saspy #SAS출력 코드

winlocal 입력

## 10장

In [84]:
%%SAS

LIBNAME ex 'C:\Biostat';
RUN;

/*10장 연습문제 불러오기*/
%MACRO chap10(name=,no=);
%do i=1 %to &no.;
	PROC IMPORT DBMS=excel
		DATAFILE="C:\Biostat\data_chap10"
		OUT=ex.&name.&i. REPLACE;
		RANGE="exam10_&i.$";
	RUN;
%end;
%MEND;

%chap10(name=ex10_,no=13);

### EXAMPLE 10.1

In [85]:
%%SAS

DATA exam_10_1;
    SET ex.ex10_1;
    if weight=. then delete;
RUN;

/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_1 normal plot;
	CLASS diet;
	VAR weight;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_1 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*등분산성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_1;
	CLASS diet;
	MODEL weight=diet/p;
	MEANS diet / HOVTEST=BARTLETT;
	ods output Bartlett = Bartlett;
RUN;
ods graphics on;ods exclude none;ods results;


title "ex10_1 : 등분산 가정 (Bartlett's Test for Homogeneity of weight Variance)";
PROC PRINT DATA=Bartlett label;
RUN;
title;

/*독립성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_1;
	CLASS diet;
	MODEL weight=diet/p;
	output out=out_ds r=resid_var;
RUN;

DATA out_ds; 
	SET out_ds;
		time=_n_;
	ods graphics on;ods exclude none;ods results;
	PROC GPLOT DATA=out_ds;
		PLOT resid_var * time;
RUN;

/*anova*/
ods graphics off;ods exclude all;ods noresults;
PROC ANOVA data=ex.ex10_1;
	class diet;
	model weight=diet;
	means diet/ TUKEY hovtest=levene(type=abs);
    ods output OverallANOVA = OverallANOVA;
    ods output CLDiffs=CLDiffs;
run;
ods graphics on;ods exclude none;ods results;
PROC PRINT DATA=OverallANOVA; 
RUN;
PROC PRINT DATA=CLDiffs; 
RUN;

OBS,VarName,diet,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,weight,1,Shapiro-Wilk,W,0.933155,Pr < W,,0.6180
2,weight,2,Shapiro-Wilk,W,0.94389,Pr < W,,0.6936
3,weight,3,Shapiro-Wilk,W,0.952018,Pr < W,,0.7287
4,weight,4,Shapiro-Wilk,W,0.990268,Pr < W,,0.9806
5,weight,1,Kolmogorov-Smirnov,D,0.208622,Pr > D,>,0.1500
6,weight,2,Kolmogorov-Smirnov,D,0.2016,Pr > D,>,0.1500
7,weight,3,Kolmogorov-Smirnov,D,0.206041,Pr > D,>,0.1500
8,weight,4,Kolmogorov-Smirnov,D,0.145566,Pr > D,>,0.1500
9,weight,1,Cramer-von Mises,W-Sq,0.035311,Pr > W-Sq,>,0.2500
10,weight,2,Cramer-von Mises,W-Sq,0.033613,Pr > W-Sq,>,0.2500


In [86]:
%%SAS

PROC IML;
USE exam_10_1;
READ all;
CLOSE exam_10_1;

k= ncol(unique(diet));
N_total=nrow(id);
sum_group=repeat(0, k);
group_mean=repeat(0, k);
group_n=repeat(0, k);
%MACRO loc(group);
%do i=1 %to 4 %by 1;
feed&i. = loc( &group= &i);
sum_group[&i.] = sum(weight[feed&i.]);
group_mean[&i.] = mean(weight[feed&i.]);
group_n[&i.]=nrow(weight[feed&i.]);
%end;
%MEND;
%loc(diet);

*(a);
sum_total = sum(sum_group);
mean_total = round(sum_total/nrow(weight), 0.0001);
mean_vec = repeat(mean_total, N_total);
totalSS = round((weight-mean_vec)`*(weight-mean_vec), 0.0001);
totalDF= N_total-1;
groupSS_el = repeat(0,k);
%MACRO groupSS_el;
%do i=1 %to 4 %by 1;
groupSS_el[&i.] = round(group_n[&i.]*((group_mean[&i.]-mean_total)**2), 0.0001);
group_mean_vec&i. = repeat(group_mean[&i.], group_n[&i.])`;
%end;
%MEND;
%groupSS_el;
groupSS = sum(groupSS_el);
groupDF=k-1;
group_mean_vec=(group_mean_vec1 || group_mean_vec2 || group_mean_vec3 ||group_mean_vec4)`;
SSE = (weight-group_mean_vec)`*(weight-group_mean_vec);
within_DF =sum(group_n-repeat(1,k));


*(b);
group_SS_first_el=repeat(0,k);
%MACRO group_SS_first;
%do i=1 %to 4 %by 1;
group_SS_first_el[&i.] = (sum_group[&i.]**2)/group_n[&i.];
%end;
%MEND;
%group_SS_first;
group_SS_first = sum(group_SS_first_el);
sum_weight = sum(weight);
sum_weightSquare = weight`*weight;

totalDF =  N_total-1;
groupsDF=k-1;
errorDF=N_total-k;

C=(sum_weight**2)/N_total;
b_totalSS = round(sum_weightSquare - C, 0.0001);
b_groupSS = round(group_SS_first - C, 0.0001);
b_errorSS = b_totalSS - b_groupSS;

*(c);
groupsMS = round( b_groupSS / groupsDF, 0.0001);
errorMS = round(b_errorSS/errorDF, 0.0001);
F = round(groupsMS /errorMS, 0.01);
critical_value = round(quantile('F', 1-0.05, groupsDF, errorDF), 0.01);
p_value = round(1- cdf('F', F, groupsDF, errorDF), 0.00001);

create exam10_1a_result var {totalSS totalDF groupSS groupDF SSE within_DF};
append; 
close exam10_1a_result;

create exam10_1b_result var {b_totalSS b_groupSS b_errorSS};
append; 
close exam10_1b_result;

create exam10_1c_result var {groupsMS errorMS F critical_value p_value};
append; 
close exam10_1c_result;

title "Example 10.1";
PRINT group_n sum_group group_mean;
RUN;
QUIT;

group_n,sum_group,group_mean
5,323.1,64.62
5,356.5,71.3
4,293.4,73.35
5,316.2,63.24


### EXAMPLE 10.1a

In [87]:
%%SAS

title "Example 10.1 (a)";
PROC PRINT DATA=exam10_1a_result; 
RUN;

OBS,TOTALSS,TOTALDF,GROUPSS,GROUPDF,SSE,WITHIN_DF
1,479.687,18,338.937,3,140.75,15


### EXAMPLE 10.1b

In [88]:
%%SAS

title "Example 10.1 (b)";
PROC PRINT DATA=exam10_1b_result; 
RUN;

OBS,B_TOTALSS,B_GROUPSS,B_ERRORSS
1,479.687,338.937,140.75


### EXAMPLE 10.1c

In [89]:
%%SAS


title "Example 10.1 (c)";

%MACRO loc(data, group, y_var);
%do i=1 %to 4 %by 1;
group&i. = loc( &group= &i);
sum_group[&i.] = sum(&y_var[group&i.]);
group_mean[&i.] = mean(&y_var[group&i.]);
group_n[&i.]=nrow(&y_var[group&i.]);
%end;
%MEND;

%MACRO group_SS_first;
%do i=1 %to 4 %by 1;
group_SS_first_el[&i.] = (sum_group[&i.]**2)/group_n[&i.];
%end;
%MEND;

%MACRO anova(data, group, y_var, SSdec, pvaluedec);
PROC IML;
USE &data;
READ all;
CLOSE &data;

k= ncol(unique(&group));
N=nrow(id);
sum_group=repeat(0, k);
group_mean=repeat(0, k);
group_n=repeat(0, k);
%loc(&data, &group, &y_var);
sum_&y_var. = sum(&y_var);
sumSquare_&y_var. = &y_var`*&y_var;
C = (sum_&y_var. **2)/N;
totalSS = round( sumSquare_&y_var. - C, 0.0001);

group_SS_first_el=repeat(0,k);
%group_SS_first;
group_SS_first = sum(group_SS_first_el);
groupsSS = group_SS_first - C;
errorSS = totalSS - groupsSS;

totalDF =  N-1;
groupsDF = k-1;
errorDF = N-k;

groupsMS = round( groupsSS  / groupsDF, &SSdec);
errorMS = round(errorSS/errorDF, &SSdec);
F = round(groupsMS /errorMS, 0.01);
critical_value = round(quantile('F', 1-0.05, groupsDF, errorDF), 0.01);
p_value = round(1- cdf('F', F, groupsDF, errorDF), &pvaluedec);

PRINT F critical_value p_value;

RUN;
QUIT;
%MEND;

%anova(exam_10_1, diet, weight, 0.0001, 0.00001);

F,critical_value,p_value
12.04,3.29,0.00028


### EXAMPLE 10.2

In [90]:
%%SAS

/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_2 normal plot;
	CLASS technician;
	VAR phosphorus;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_2 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*독립성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_2;
	CLASS technician;
	MODEL phosphorus=technician/p;
	output out=out_ds r=resid_var;
RUN;
DATA out_ds; 
	SET out_ds;
	time=_n_;
	ods graphics on;ods exclude none;ods results;
PROC GPLOT DATA=out_ds;
	PLOT resid_var * time;
RUN;

/*등분산성*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_2 plots=(DIAGNOSTICS RESIDUALS);
	CLASS technician;
	MODEL phosphorus=technician;
	MEANS technician / HOVTEST=LEVENE(type=abs) hovtest=bf;
	ods output HOVFTest = HOVFTest;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_2 : 등분산 가정 (Levene's Test & Brown and Forsythe's Test )";
PROC PRINT DATA=HOVFTest label;
RUN;
title;

/*KruskalWallisTest*/
ods graphics off;ods exclude all;ods noresults;
PROC NPAR1WAY DATA=ex.ex10_2 wilcoxon;
	ods trace on;
	CLASS technician;
	VAR phosphorus;
	ods output KruskalWallisTest=KruskalWallisTest;
RUN;
ods graphics on;ods exclude none;ods results;
title "ex10_2 : KruskalWallisTest";
PROC PRINT DATA=KruskalWallisTest label; 
RUN;

/*혼합모형  anova*/
ods graphics off;ods exclude all;ods noresults;
PROC MIXED DATA=ex.ex10_2  COVTEST CL;
	CLASS technician ;
	MODEL phosphorus= ;
	RANDOM technician  ; 
	ods output CovParms=CovParms;
RUN; 
QUIT; 
ods graphics on;ods exclude none;ods results;
title "ex10_2 : Covariance Parameter Estimates";
PROC PRINT DATA=CovParms label;
RUN;
title;

OBS,VarName,technician,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,phosphorus,1,Shapiro-Wilk,W,0.770908,Pr < W,,0.0460
2,phosphorus,2,Shapiro-Wilk,W,0.770908,Pr < W,,0.0460
3,phosphorus,3,Shapiro-Wilk,W,0.90202,Pr < W,,0.4211
4,phosphorus,4,Shapiro-Wilk,W,0.90202,Pr < W,,0.4211
5,phosphorus,1,Kolmogorov-Smirnov,D,0.348833,Pr > D,,0.0441
6,phosphorus,2,Kolmogorov-Smirnov,D,0.348833,Pr > D,,0.0441
7,phosphorus,3,Kolmogorov-Smirnov,D,0.221307,Pr > D,>,0.1500
8,phosphorus,4,Kolmogorov-Smirnov,D,0.221307,Pr > D,>,0.1500
9,phosphorus,1,Cramer-von Mises,W-Sq,0.10627,Pr > W-Sq,,0.0707
10,phosphorus,2,Cramer-von Mises,W-Sq,0.10627,Pr > W-Sq,,0.0707


### EXAMPLE 10.3

In [91]:
%%SAS
/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_3 normal plot;
	CLASS variety;
	VAR potassium;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_3 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*welch's anova*/
ods graphics off;ods exclude all;ods noresults;
PROC ANOVA data=ex.ex10_3;
	CLASS variety;
	MODEL potassium=variety;
	MEANS variety/  HOVTEST=BARTLETT welch;
	ods output Bartlett = Bartlett;
	ods output welch=welch;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_3 : 등분산 가정 (Bartlett's Test for Homogeneity of weight Variance)";
PROC PRINT DATA=Bartlett;
RUN;
title;

title "ex10_3 : Welch's test";
PROC PRINT DATA=welch; 
RUN;
title;

ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_3 ;
	CLASS variety;
	VAR potassium;
	ods output moments=moments;
RUN;
ods graphics on;ods exclude none;ods results;
PROC PRINT DATA=moments; 
RUN;

OBS,VarName,variety,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,potassium,1,Shapiro-Wilk,W,0.977796,Pr < W,,0.9401
2,potassium,2,Shapiro-Wilk,W,0.966512,Pr < W,,0.8682
3,potassium,3,Shapiro-Wilk,W,0.968814,Pr < W,,0.8844
4,potassium,1,Kolmogorov-Smirnov,D,0.176451,Pr > D,>,0.1500
5,potassium,2,Kolmogorov-Smirnov,D,0.184113,Pr > D,>,0.1500
6,potassium,3,Kolmogorov-Smirnov,D,0.151176,Pr > D,>,0.1500
7,potassium,1,Cramer-von Mises,W-Sq,0.028922,Pr > W-Sq,>,0.2500
8,potassium,2,Cramer-von Mises,W-Sq,0.032192,Pr > W-Sq,>,0.2500
9,potassium,3,Cramer-von Mises,W-Sq,0.023808,Pr > W-Sq,>,0.2500
10,potassium,1,Anderson-Darling,A-Sq,0.17631,Pr > A-Sq,>,0.2500


### EXAMPLE 10.4

In [92]:
%%SAS

PROC POWER;
	onewayanova
	groupmeans=8.0 |8.0|9.0|12.0
	stddev=2.754778
	alpha=0.05
	npergroup=10
    power=. ;
RUN;

PROC IML;
k=4;
n=10;
v1 = k-1;
v2 = k*(n-1);
means_i = {8.0 8.0 9.0 12.0}`;
mu = mean(means_i);
sigma_2=7.5888;
phi = round(sqrt((n*((means_i - repeat(mu, 4))`*(means_i - repeat(mu, 4))))/(k*sigma_2)), 0.01);

PRINT phi v1 v2 ;
RUN;
QUIT;

### EXAMPLE 10.5

In [93]:
%%SAS

PROC IML;
	reset nolong;
	k=4;d=4;s=7.5888;n=10;
	p=sqrt((n*d*d)/(2*k*s));
	print p;
RUN;
QUIT;

/*결과 p=1.5645802*/

p
1.6234108


### EXAMPLE 10.6

In [94]:
%%SAS

PROC IML;
	reset nolong;
	k=4;d=3.5;s=9.383;n=15;
	p=sqrt((n*d*d)/(2*k*s));
	PRINT p;
RUN;
QUIT;

PROC IML;
	reset nolong;
	k=4;d=3.5;s=9.383;n=20;
	p=sqrt((n*d*d)/(2*k*s));
	PRINT p;
RUN;
QUIT;

PROC IML;
	reset nolong;
	k=4;d=3.5;s=9.383;n=18;
	p=sqrt((n*d*d)/(2*k*s));
	print p;
RUN;
QUIT;

/*결과 p=1.7139117*/

p
1.5645802
p
1.8066216
p
1.7139117


### EXAMPLE 10.7

In [95]:
%%SAS

PROC IML;
	reset nolong;
	k=4;p=2;s=9.383;n=10;
	d=sqrt((2*k*s*(p*p))/n);
	print d;
RUN;
QUIT;

d
5.479562


### EXAMPLE 10.8

In [96]:
%%SAS

PROC IML;
	reset nolong;
	d=4.5;s=9.383;
	p=sqrt((8*d*d)/(2*6*s));/*k=6라 가정하고 시행*/
	print p;
RUN;
QUIT;

PROC IML;
	reset nolong;
	d=4.5;s=9.383;
	p=sqrt((10*d*d)/(2*5*s));/*k=5라 가정하고 시행*/
	print p;
RUN;
QUIT;

PROC IML;
	reset nolong;
	d=4.5;s=9.383;
	p=sqrt((12*d*d)/(2*4*s));/*k=4라 가정하고 시행*/
	print p;
RUN;
QUIT;

/*k=4일때 power=0.80임을 알수있음*/

p
1.1994883
p
1.4690672
p
1.7992325


### EXAMPLE 10.9

In [97]:
%%SAS

PROC IML;
	reset nolong;
	gr_ms=3;s=1.25;v1=3;v2=16;
	F1=finv(1-0.05, v1,v2);
   F2=(v2*s*F1)/((v2-2)*(gr_ms));
   p=1-probF(F2,v1,v2);
   print F1 F2 p;
RUN;
QUIT;

F1,F2,p
3.2388715,1.5423198,0.2421396


### EXAMPLE 10.10

In [98]:
%%SAS

/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_10 normal plot;
	CLASS layer;
	VAR abundance;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_10 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*등분산성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_10;
	CLASS layer;
	MODEL abundance=layer/p;
	MEANS layer / HOVTEST=BARTLETT;
	ods output Bartlett = Bartlett;
RUN;
ods graphics on;ods exclude none;ods results;


title "ex10_10 : 등분산 가정 (Bartlett's Test for Homogeneity of weight Variance)";
PROC PRINT DATA=Bartlett label;
RUN;
title;

/*anova*/
ods graphics off;ods exclude all;ods noresults;
PROC ANOVA DATA=ex.ex10_10;
	CLASS layer;
	MODEL abundance = layer;
	MEANS layer/ TUKEY hovtest=levene(type=abs);
	ods output OverallANOVA = OverallANOVA;
RUN;
ods graphics on;ods exclude none;ods results;
PROC PRINT DATA=OverallANOVA; 
RUN;
QUIT;

/*Kruskal-Wallis Test*/
ods graphics off;ods exclude all;ods noresults;
PROC NPAR1WAY DATA=ex.ex10_10;
	CLASS layer;
	VAR abundance;
	ods output KruskalWallisTest=KruskalWallisTest;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_10 : Kruskal-Wallis Test";
PROC PRINT DATA=KruskalWallisTest; 
RUN;
title;
QUIT;


OBS,VarName,layer,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,abundance,1,Shapiro-Wilk,W,0.970154,Pr < W,,0.8762
2,abundance,2,Shapiro-Wilk,W,0.923293,Pr < W,,0.5514
3,abundance,3,Shapiro-Wilk,W,0.958909,Pr < W,,0.8004
4,abundance,1,Kolmogorov-Smirnov,D,0.207939,Pr > D,>,0.1500
5,abundance,2,Kolmogorov-Smirnov,D,0.231739,Pr > D,>,0.1500
6,abundance,3,Kolmogorov-Smirnov,D,0.184327,Pr > D,>,0.1500
7,abundance,1,Cramer-von Mises,W-Sq,0.029496,Pr > W-Sq,>,0.2500
8,abundance,2,Cramer-von Mises,W-Sq,0.042843,Pr > W-Sq,>,0.2500
9,abundance,3,Cramer-von Mises,W-Sq,0.028686,Pr > W-Sq,>,0.2500
10,abundance,1,Anderson-Darling,A-Sq,0.189383,Pr > A-Sq,>,0.2500


### EXAMPLE 10.11

In [99]:
%%SAS

/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_11 normal plot;
	CLASS pond;
	VAR ph;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_11 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*등분산성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_11;
	CLASS pond;
	MODEL ph=pond/p;
	MEANS pond / HOVTEST=BARTLETT;
	ods output Bartlett = Bartlett;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_11 : 등분산 가정 (Bartlett's Test for Homogeneity of weight Variance)";
PROC PRINT DATA=Bartlett label;
RUN;
title;

/*Kruskal-WallisTest*/
ods graphics off;ods exclude all;ods noresults;
PROC NPAR1WAY DATA=ex.ex10_11;
	CLASS pond;
	VAR ph;
	ods output KruskalWallisTest=KruskalWallisTest;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_11 : Kruskal-Wallis Test";
PROC PRINT DATA=KruskalWallisTest; 
RUN;
title;
QUIT;

/*welch's anova*/
ods graphics off;ods exclude all;ods noresults;
PROC ANOVA data=ex.ex10_11;
	CLASS pond;
	MODEL ph=pond ;
	MEANS pond/  welch;
	ods output welch=welch;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_11 : Welch's test";
PROC PRINT DATA=welch; 
RUN;
title;

OBS,VarName,pond,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,ph,1,Shapiro-Wilk,W,0.950005,Pr < W,,0.7113
2,ph,2,Shapiro-Wilk,W,0.929589,Pr < W,,0.5123
3,ph,3,Shapiro-Wilk,W,0.97382,Pr < W,,0.9245
4,ph,4,Shapiro-Wilk,W,0.933682,Pr < W,,0.5502
5,ph,1,Kolmogorov-Smirnov,D,0.200477,Pr > D,>,0.1500
6,ph,2,Kolmogorov-Smirnov,D,0.222329,Pr > D,>,0.1500
7,ph,3,Kolmogorov-Smirnov,D,0.121718,Pr > D,>,0.1500
8,ph,4,Kolmogorov-Smirnov,D,0.157956,Pr > D,>,0.1500
9,ph,1,Cramer-von Mises,W-Sq,0.040945,Pr > W-Sq,>,0.2500
10,ph,2,Cramer-von Mises,W-Sq,0.058881,Pr > W-Sq,>,0.2500


### EXAMPLE 10.12

In [100]:
%%SAS

/*정규성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex10_12 normal plot;
	CLASS side;
	VAR height;
	ods output TestsForNormality = TestsForNormality;
RUN;
ods graphics on;ods exclude none;ods results;
PROC SORT DATA=TestsForNormality;
	BY descending Test;
RUN;
title " ex10_12 : 정규성 가정";
PROC PRINT DATA=TestsForNormality label;
RUN;
title;

/*등분산성 검정*/
ods graphics off;ods exclude all;ods noresults;
PROC GLM DATA=ex.ex10_12;
	CLASS side;
	MODEL height=side/p;
	MEANS side/ HOVTEST=BARTLETT;
	ods output Bartlett = Bartlett;
RUN;
ods graphics on;ods exclude none;ods results;

title "ex10_12 : 등분산 가정 (Bartlett's Test for Homogeneity of weight Variance)";
PROC PRINT DATA=Bartlett label;
RUN;
title;

PROC NPAR1WAY DATA=ex.ex10_12 median correct=no;
	CLASS side;
	VAR height;
RUN;

DATA e_1012;
   INPUT median $ side $ ex_f;
CARDS;
above north 4
above east 3
above south 10 
above west 6
below north 7
below east 9
below south 1
below west 6
;
RUN;

PROC FREQ DATA=e_1012 order=data;
    WEIGHT ex_f;
    TABLES median * side / nopercent nocol norow chisq;
RUN;

OBS,VarName,side,적합도 검정,적합도 통계량에 대한 레이블,적합도 통계량 값,p-값 레이블,p-값의 부호,p-값
1,height,1,Shapiro-Wilk,W,0.953812,Pr < W,,0.6932
2,height,2,Shapiro-Wilk,W,0.919429,Pr < W,,0.2812
3,height,3,Shapiro-Wilk,W,0.933638,Pr < W,,0.4203
4,height,4,Shapiro-Wilk,W,0.898768,Pr < W,,0.1529
5,height,1,Kolmogorov-Smirnov,D,0.183335,Pr > D,>,0.1500
6,height,2,Kolmogorov-Smirnov,D,0.189739,Pr > D,>,0.1500
7,height,3,Kolmogorov-Smirnov,D,0.228172,Pr > D,,0.0835
8,height,4,Kolmogorov-Smirnov,D,0.161488,Pr > D,>,0.1500
9,height,1,Cramer-von Mises,W-Sq,0.046571,Pr > W-Sq,>,0.2500
10,height,2,Cramer-von Mises,W-Sq,0.070453,Pr > W-Sq,>,0.2500


### EXAMPLE 10.13

In [101]:
%%SAS

ods graphics off;ods exclude all;ods noresults;
PROC ANOVA DATA=exam_10_1;
	CLASS diet;
	MODEL weight=diet;
	MEANS diet/  hovtest= bartlett;
	ods output Bartlett=Bartlett;
RUN;
ods graphics on;ods exclude none;ods results;
PROC PRINT DATA=Bartlett;
RUN;

OBS,Effect,Dependent,Source,DF,ChiSq,ProbChiSq
1,diet,weight,diet,3,0.4752,0.9243


### EXAMPLE 10.14

In [102]:
%%SAS

PROC IML;
n={5, 5, 4, 5};
v={4, 4, 3, 4};
v_a={0.0518, 0.0597, 0.0465, 0.0456};
sum_v=sum(v);
sum_vv=sum(v#v_a);
v_p=sum_vv/sum_v;
v_p2=(v_p)**2;
sum_vv2=sum((v#v_a)#v_a);
chi_stat=round((sum_vv2-(((sum_vv)**2)/sum_v))/(v_p2*(0.5+v_p2)), 0.001);
chi_crit=round(abs(quantile("chisq", 1-0.05, 3)), 0.001);
PRINT sum_v sum_vv v_p v_p2 sum_vv2 chi_stat chi_crit;
RUN;
QUIT;

sum_v,sum_vv,v_p,v_p2,sum_vv2,chi_stat,chi_crit
15,0.7679,0.0511933,0.0026208,0.0397935,0.366,7.815
